In [11]:
task = "drugrec"

In [12]:
import pickle

with open(f'/data/pj20/exp_data/ccscm_ccsproc/sample_dataset_mimic3_{task}_th015.pkl', 'rb') as f:
    sample_dataset = pickle.load(f)

In [13]:
len(sample_dataset)

44399

In [3]:
from pyhealth.datasets import split_by_patient, get_dataloader

train_dataset, val_dataset, test_dataset = split_by_patient(sample_dataset, [0.8, 0.1, 0.1], train_ratio=1.0, seed=528)

In [4]:
train_dataset = list(train_dataset)

In [9]:
import random
from collections import defaultdict

def nested_dict():
    return defaultdict(list)

def drop_ratio(d, ratio):
    new_d = defaultdict(list)
    for key, value in d.items():
        if key == "conditions" or key == "procedures" or key == "drugs":
            if isinstance(value[0], list):
                for i in range(len(value)):
                        n = int(len(value[i]) * (ratio))  # calculate the number of items to keep
                        sampled = random.sample(value[i], n)  # randomly select items to keep
                        if len(sampled) > 0:
                            new_d[key].append(sampled)
                        else:
                            new_d[key].append(random.sample(value[i], 1))
            else:
                n = int(len(value) * (ratio))
                sampled = random.sample(value, n)
                if len(new_d[key]) == 0:
                    new_d[key] = value
                else:
                    new_d[key] = random.sample(value, 1)
        elif key == "label":
            new_d[key] = value
        else:
            new_d[key] = value
    return new_d

In [8]:
ratios = [
    0.05,
    0.1,
    0.2,
    0.3,
    0.4,
    0.5,
    0.6,
    0.7,
    0.8,
    0.9,
]

In [10]:
from copy import deepcopy
from data_prepare import *
from tqdm import tqdm

for ratio in ratios:
    print("ratio: ", ratio)
    train_instance = deepcopy(train_dataset)

    for i in range(len(train_instance)):
        train_instance[i] = drop_ratio(train_instance[i], ratio)

    print("Loading embeddings...")
    ent2id, rel2id, ent_emb, rel_emb = load_embeddings(task)
    if task == "drugrec":
        train_instance = prepare_drug_indices(train_instance)

    map_cluster, map_cluster_inv, map_cluster_rel, map_cluster_inv_rel = clustering(task, ent_emb, rel_emb, threshold=0.15, load_cluster=True, save_cluster=False)

    print("Processing graph...")
    G = process_graph("mimic3", task, train_instance, ent2id, rel2id, map_cluster, map_cluster_inv, map_cluster_rel, map_cluster_inv_rel, save_graph=False)
    G_tg = from_networkx(G)
    
    print("Processing dataset...")
    train_instance = process_sample_dataset("mimic3", task, train_instance, G_tg, ent2id, rel2id, map_cluster, map_cluster_inv, map_cluster_rel, map_cluster_inv_rel, save_dataset=False)

    with open(f'/data/pj20/exp_data/ccscm_ccsproc_atc3/train_dataset_mimic3_{task}_th015_{ratio}.pkl', 'wb') as f:
        pickle.dump(train_instance, f)

    print(f"Done ratio: {ratio}")

ratio:  0.05
Loading embeddings...
Processing graph...


100%|██████████| 7730/7730 [00:07<00:00, 1043.27it/s]


Processing dataset...


100%|██████████| 7730/7730 [00:07<00:00, 979.27it/s] 


Done ratio: 0.05
ratio:  0.1
Loading embeddings...
Processing graph...


...

Done ratio: 0.8
ratio:  0.9
Loading embeddings...
Processing graph...


100%|██████████| 7730/7730 [01:50<00:00, 70.09it/s]


Processing dataset...


100%|██████████| 7730/7730 [01:21<00:00, 95.06it/s] 


Done ratio: 0.9
